<a href="https://colab.research.google.com/github/Mmbsaksd/ANN-classification-on-churn-data/blob/main/Prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [1]:
import tensorflow as tf
from tensorflow.keras.models import load_model
import pickle
import pandas as pd
import numpy as np

In [5]:
#Load ANN train model, scaler pickle file, onehot
model = load_model('/content/drive/MyDrive/Complete Gen AI Krish Udemy/annclassification/files/model.h5')

#Load the encoder and scaler
with open('/content/drive/MyDrive/Complete Gen AI Krish Udemy/annclassification/files/label_encoder_gender.pkl','rb') as file:
  label_encoder_gender = pickle.load(file)

with open('/content/drive/MyDrive/Complete Gen AI Krish Udemy/annclassification/files/onehot_encoder_geo.pkl','rb') as file:
  onehot_encoder_geo = pickle.load(file)

with open('/content/drive/MyDrive/Complete Gen AI Krish Udemy/annclassification/files/scaler.pkl','rb') as file:
  scaler = pickle.load(file)

In [10]:
# Example input data
input_data = {
    'CreditScore': 600,
    'Geography': 'France',
    'Gender': 'Male',
    'Age': 40,
    'Tenure': 3,
    'Balance': 60000,
    'NumOfProducts': 2,
    'HasCrCard': 1,
    'IsActiveMember': 1,
    'EstimatedSalary': 50000
}

In [11]:
geo_encoded = onehot_encoder_geo.transform([[input_data['Geography']]]).toarray()
geo_encoded_df = pd.DataFrame(geo_encoded, columns=onehot_encoder_geo.get_feature_names_out(['Geography']))
geo_encoded_df

/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but OneHotEncoder was fitted with feature names
  warnings.warn(


,Geography_France,Geography_Germany,Geography_Spain
0,1.0,0.0,0.0


In [14]:
input_data_df = pd.DataFrame([input_data])
input_data_df

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,600,France,Male,40,3,60000,2,1,1,50000


In [15]:
## Encode categorical variable
input_data_df['Gender'] = label_encoder_gender.transform(input_data_df['Gender'])
input_data_df

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,600,France,1,40,3,60000,2,1,1,50000


In [16]:
## Concatination one hot encoding
input_df = pd.concat([input_data_df.drop("Geography", axis=1), geo_encoded_df],axis=1)
input_df

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Geography_France,Geography_Germany,Geography_Spain
0,600,1,40,3,60000,2,1,1,50000,1.0,0.0,0.0


In [17]:
## Scaling the input data
input_scaled = scaler.transform(input_df)
input_scaled

array([[-0.47154541,  0.90911166,  0.09477172, -0.69844549, -0.29010416,
         0.80510537,  0.63367318,  0.95214374, -0.84805047,  0.98019606,
        -0.57581067, -0.56349184]])

In [18]:
## Prediction churn
prediction = model.predict(input_scaled)
prediction

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 145ms/step


array([[0.04462901]], dtype=float32)

In [19]:
prediction_prob = prediction[0][0]
if prediction_prob > 0.5:
  print("Customer is likely to churn")
else:
  print("Customer is not likely churn")

Customer is not likely churn
